<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_11_05_english_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks
**Module 11: Natural Language Processing and Speech Recognition**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

* Part 11.1: Getting Started with Spacy in Python [[Video]](https://www.youtube.com/watch?v=A5BtU9vXzu8&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_01_spacy.ipynb)
* Part 11.2: Word2Vec and Text Classification [[Video]](https://www.youtube.com/watch?v=nWxtRlpObIs&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_02_word2vec.ipynb)
* Part 11.3: What are Embedding Layers in Keras [[Video]](https://www.youtube.com/watch?v=OuNH5kT-aD0&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_03_embedding.ipynb)
* Part 11.4: Natural Language Processing with Spacy and Keras [[Video]](https://www.youtube.com/watch?v=BKgwjhao5DU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_04_text_nlp.ipynb)
* **Part 11.5: Learning English from Scratch with Keras and TensorFlow** [[Video]](https://www.youtube.com/watch?v=Y1khuuSjZzc&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=58) [[Notebook]](t81_558_class_11_05_english_scratch.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


# Part 11.5: Learning English from Scratch with Keras and TensorFlow

In this section we will see how a neural network can learn the English language from scratch.  We will make use of a type of neural network called end-to-end memory. [[Cite:sukhbaatar2015end]](https://arxiv.org/abs/1503.08895) We will train this type of neural network on a special dataset that was created by researchers at Facebook to test a neural network's ability to answer questions. [[Cite:sukhbaatar2015end]](https://arxiv.org/pdf/1502.05698.pdf) 

Other useful links for End-To-End Memory Networks

* [bAbI Datasets](https://research.fb.com/downloads/babi/) - The Facebook dataset used to train this network.
* [Keras End-To-End Memory Networks](https://github.com/fchollet/keras/blob/master/examples/babi_memnn.py) - Example from Keras author on end-to-end networks.
* [Online JavaScript Demo of End-to-End Memory Networks](http://yerevann.com/dmn-ui/#/)

## Imports and Utility Functions

The following imports are needed to create the end-to-end memory network. Neither Keras nor TensorFlow directly supports End-to-End Memory Networks (yet), so it is necessary to develop them using existing tools.  Several functions are needed to be defined here to read the bAbI dataset that we are using to train.


In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input, Activation, Dense, Permute
from tensorflow.keras.layers import Dropout, add, dot, concatenate
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from functools import reduce
import pickle
import tarfile
import numpy as np
import re
import os
import time

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"


def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the',\ 
        'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data \
            if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        answers.append(word_idx[answer])
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))


## Getting the Data

The data is first downloaded from the Internet if needed. This dataset contains stories and questions about those stories.  The computer is not learning these specific stories.  However, it is learning how to read a story and answer a question about that story.  Consider the first story, "Mary moved to the bathroom. John went to the hallway." the computer is not learning that Mary is in the bathroom or John is in the hallway, this changes per story. Instead, the machine is learning to parse the story and extract information about individual people and their locations.

The computer is learning to read, at least in a limited sense.

In [3]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', 
        origin='https://s3.amazonaws.com/text-datasets/"\
            "babi_tasks_1-20_v1-2.tar.gz')
except:
    print("""
Error downloading dataset, please download it manually:\n'
$ wget http://www.thespermwhale.com/jaseweston/babi/
    tasks_1-20_v1-2.tar.gz\n'
$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/
    babi-tasks-v1-2.tar.gz""")
    raise
tar = tarfile.open(path)

challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 
        'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 
        'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))



Extracting stories for the challenge: single_supporting_fact_10k


In [4]:
# See what the data looks like

for i in range(5):
    print("Story: {}".format(' '.join(train_stories[i][0])))
    print("Query: {}".format(' '.join(train_stories[i][1])))
    print("Answer: {}".format(train_stories[i][2]))
    print("---")

Story: Mary moved to the bathroom . John went to the hallway .
Query: Where is Mary ?
Answer: bathroom
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden .
Query: Where is Daniel ?
Answer: hallway
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom .
Query: Where is Daniel ?
Answer: hallway
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . Mary moved to the hallway . Daniel travelled to the office .
Query: Where is Daniel ?
Answer: office
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . Mary mov

## Building the Vocabulary

This type of neural network can only deal with a set vocabulary.  The words are indexed, and each becomes a number.  Words not in the training vocabulary will not be recognized.

In [5]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')


for s in list(enumerate(vocab)):
    print(s)

-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
(0, '.')
(1, '?')
(2, 'Daniel')
(3, 'John')
(4, 'Mary')
(5, 'Sandra')
(6, 'Where')
(7, 'back')
(8, 'bathroom')
(9, 'bedroom')
(10, 'garden')
(11, 'hallway')
(12, 'is')
(13, 'journeyed')
(14, 'kitchen')
(15, 'moved')
(16, 'office')
(17, 'the')
(18, 'to')
(19, 'travelled')
(20, 'went')


## Building the Training and Test Data

We present the training data to the neural network as a vectorized representation of the sentences.  We replace each word with that word's corresponding vocabulary index.  Additionally, there are two parts to the input (x) data: story and query.  The answer (x) is always a single vocab word number.  We set up this neural network for classification.  Any of the vocab words could potentially be the answer.  Stories can be at most 68 words and questions at most 4.  The program automatically determines both of these limits from the training data.

In [6]:
print('Vectorizing the word sequences...')
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train \
    = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test \
    = vectorize_stories(test_stories)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')


Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000,)
answers_test shape: (1000,)
-


In [7]:
# See individual training element.

print("Story (x): {}".format(inputs_train[0]))
print("Question (x): {}".format(queries_train[0]))
print("Answer: {}".format(answers_train[0]))

Story (x): [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  5 16 19 18  9  1  4 21 19 18 12  1]
Question (x): [ 7 13  5  2]
Answer: 9


## Compile the Neural Network

We define the structure of the neural network according to the end-to-end structure defined in the paper discussed previously. 

In [8]:
print('Compiling...')

# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  
# (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  
# (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print("Done.")


Compiling...
Done.


## Train the Neural Network

It will take some time (probably up to 1/2 hour) to train this network on a CPU.  Once complete, the program saves the network.  If you've previously saved the neural network, you can skip this step and load it in the next step.

In [9]:
start_time = time.time()
# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

# save
save_path = "./data/"
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"chatbot.h5"))
# save the vocab too, indexes must be the same
pickle.dump( vocab, open( os.path.join(save_path,"vocab.pkl"), "wb" ) )

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 8s 833us/sample - loss: 1.9471 - accuracy: 0.1651 - val_loss: 1.7972 - val_accuracy: 0.1810
Epoch 2/120
10000/10000 [==============================] - 3s 264us/sample - loss: 1.7601 - accuracy: 0.2340 - val_loss: 1.6338 - val_accuracy: 0.3360
Epoch 3/120
10000/10000 [==============================] - 3s 266us/sample - loss: 1.5809 - accuracy: 0.3713 - val_loss: 1.5326 - val_accuracy: 0.3750
Epoch 4/120
10000/10000 [==============================] - 3s 269us/sample - loss: 1.5284 - accuracy: 0.3880 - val_loss: 1.4769 - val_accuracy: 0.4120
Epoch 5/120
10000/10000 [==============================] - 3s 264us/sample - loss: 1.4962 - accuracy: 0.4024 - val_loss: 1.4577 - val_accuracy: 0.4180
Epoch 6/120
10000/10000 [==============================] - 3s 269us/sample - loss: 1.4661 - accuracy: 0.4143 - val_loss: 1.4012 - val_accuracy: 0.4370
Epoch 7/120
10000/10000 [====================

Epoch 108/120
10000/10000 [==============================] - 3s 267us/sample - loss: 0.0917 - accuracy: 0.9663 - val_loss: 0.3405 - val_accuracy: 0.9030
Epoch 109/120
10000/10000 [==============================] - 3s 272us/sample - loss: 0.0949 - accuracy: 0.9676 - val_loss: 0.1673 - val_accuracy: 0.9460
Epoch 110/120
10000/10000 [==============================] - 3s 270us/sample - loss: 0.0890 - accuracy: 0.9697 - val_loss: 0.1612 - val_accuracy: 0.9490
Epoch 111/120
10000/10000 [==============================] - 3s 264us/sample - loss: 0.0858 - accuracy: 0.9713 - val_loss: 0.1607 - val_accuracy: 0.9500
Epoch 112/120
10000/10000 [==============================] - 3s 264us/sample - loss: 0.0784 - accuracy: 0.9734 - val_loss: 0.1811 - val_accuracy: 0.9520
Epoch 113/120
10000/10000 [==============================] - 3s 270us/sample - loss: 0.0843 - accuracy: 0.9710 - val_loss: 0.1714 - val_accuracy: 0.9460
Epoch 114/120
10000/10000 [==============================] - 3s 267us/sample - los

In [10]:
# Load the model, if it exists, load vocab too
save_path = "./data/"
model = load_model(os.path.join(save_path,"chatbot.h5"))
vocab = pickle.load( open( os.path.join(save_path,"vocab.pkl"), "rb" ) )

## Evaluate Accuracy

We evaluate the accuracy, using the same technique as previous classification networks.

In [11]:
pred = model.predict([inputs_test, queries_test])
# See what the predictions look like, they are just probabilities 
# of each class.
print(pred)

[[3.2316551e-18 4.5703689e-18 4.2100677e-18 ... 2.7742484e-18
  3.3352055e-18 3.6209529e-18]
 [8.6646321e-15 8.6298309e-15 7.7134844e-15 ... 8.2590549e-15
  7.4069846e-15 9.1656156e-15]
 [1.6307067e-15 2.1523891e-15 2.2016419e-15 ... 1.6251486e-15
  2.1003965e-15 2.4207346e-15]
 ...
 [2.0678353e-16 2.2777190e-16 2.1688923e-16 ... 2.1682140e-16
  2.0552370e-16 1.9355541e-16]
 [5.6333461e-17 5.9818837e-17 4.9700267e-17 ... 5.3023650e-17
  6.0595586e-17 5.4082653e-17]
 [1.6186161e-09 1.6892178e-09 1.6479546e-09 ... 1.6192523e-09
  1.3914115e-09 1.7729884e-09]]


In [12]:
# Use argmax to turn those into actual predictions.  The class (word) 
# with the highest
# probability is the answer.

pred = np.argmax(pred,axis=1)
print(pred)

[12  9 15 12 15 12 11 12 17 17 15 11 17 10 10 15 11 15 17 12 12 17 17 12
 10 10 10 15 15 15 12 17 15 15  9 17  9 11 15 10  9 11 10 12 11 15 12  9
 17 10 10 17 11 11 12 10 15 12 11 12 12 17  9 11 10 15 15  9 17 17 11 12
 11 10  9 15 15 15 12 17 11  9 10 15 17 11 11 12 15 15  9  9 12  9  9 15
 15 15 10 10 10  9 17 11 11 12 10  9 17 10  9  9 15 12 15 11 17 15  9 17
 10  9  9 12 10 17 11 11 11  9 11 12  9 11 17  9 17 15  9  9 17 12 12 17
 10 15 15 15 15 15 11 11  9 17 10  9 11 10 17 12 10 10 10 11  9 11 11 11
 15 10 17 17 11  9  9 17 15 10 17 17 12 12 12 12 10  9 17 11 15 17 12 15
 11 12 12 11 17 15  9 10 12 17 12 10 12  9 15 12  9 12  9 10 12 10  9 10
  9 12 12  9  9 12 15 11 10 17 12 11  9  9 17 11 12 17 12 10 10  9 10 11
 12 15 11 11 15 15 11 11 11 11 10 17  9  9 10 10 17 17 15 11 11 17 12 17
 17  9 11 10  9 10 15 17 17 15  9 17 15 12 17  9  9 11 17 17 11 17  9 17
 11  9 15 15 10 15 15 11 11 12 12  9  9  9  9 17 12 11  9 17 17 15 11 11
 17 10 11 17 10 17 17 11 11 15 15 11 11  9 10 11 11

In [13]:
score = metrics.accuracy_score(answers_test, pred)
print("Final accuracy: {}".format(score))

Final accuracy: 0.956


## Adhoc Query

You might want to create your own stories and questions.  

In [14]:
print("Remember, I only know these words: {}".format(vocab))
print()
story = "Daniel went to the hallway. Mary went to the bathroom."\
    "Daniel went to the bedroom."
query = "Where is Sandra?"

adhoc_stories = (tokenize(story), tokenize(query), '?')

adhoc_train, adhoc_query, adhoc_answer = vectorize_stories([adhoc_stories])

pred = model.predict([adhoc_train, adhoc_query])
print(pred[0])
pred = np.argmax(pred,axis=1)
print("Answer: {}({})".format(vocab[pred[0]-1],pred))


Remember, I only know these words: ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']

[5.3319661e-11 5.9570487e-11 5.1706601e-11 5.1733437e-11 5.4510025e-11
 5.7622622e-11 4.8129528e-11 5.3163075e-11 5.6659514e-11 8.2092601e-01
 2.8999595e-02 1.4757804e-02 3.3609481e-03 5.6475557e-11 5.3110880e-11
 1.3155086e-01 5.3828338e-11 4.0473844e-04 5.5407737e-11 5.2006194e-11
 5.0534833e-11 5.6617922e-11]
Answer: bathroom([9])
